<a href="https://colab.research.google.com/github/Ashish-18/practice/blob/master/Copy_of_M1_MP1_NB_Data_Munging_(Ungraded).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science

##  A program by IISc and TalentSprint

### Mini Project Notebook 1 : Data Munging (Ungraded)



## Learning Objectives



At the end of the experiment, you will be able to :


* understand the requirements for a “clean” dataset, ready for use in statistical analysis

* use Python libraries like Pandas, Numpy, and Matplotlib to perform the  data-preprocessing steps

* obtain probability and statistics based insights from the data


## Dataset

The dataset chosen for this experiment is the **Play Store** dataset which is  publicly available and created with this [methodology](https://nycdatascience.com/blog/student-works/google-play-store-everything-that-you-need-to-know-about-the-android-market/)  

This dataset consists of 10841 records. Each record is made up of 13 fields.

**For example**, Each record consists of App, Category, Rating, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver, and Android Ver.

## Problem Statement

Before we can derive any meaningful insights from the Play Store data, it is essential to pre-process the data and make it suitable for further analysis. This pre-processing step forms a major part of data wrangling (or data munging) and ensures better quality data. It consists of the transformation and mapping of data from a "raw" data form into another format so that it is more valuable for a variety of downstream purposes such as analytics. Data analysts typically spend a sizeable amount of time in the process of data wrangling, compared to the actual analysis of the data.

After data munging is performed, several actionable insights can be derived from the Play Store apps data. Such insights could help to unlock the enormous potential to drive app-making businesses to success.

In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/CDS/Datasets/googleplaystore.csv

#### Import required packages

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt

#### Load the dataset

In [ ]:
# YOUR CODE HERE
df1= pd.read_csv("googleplaystore.csv")
df1.head(3)

## Pre-processing

### Task 1: Data Cleaning

* Check whether there are any null values and figure out how you want to handle them?
  
    **Hint:** isnan(), dropna(), fillna()
* If there is any duplication of a record, how would you like to handle it?

    Hint: [drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html)

* Are there any non-English apps? And how to filter them?

* In the size column, multiply 1,000,000 with M in the cell and multiply by 1000 if we have K in the cell.

# YOUR CODE HERE
df1=df1.drop_duplicates()
df1

In [ ]:
# Check whether there are any null values and figure out how you want to handle them?
df1.isna().sum()

In [ ]:
app=df1.App.unique()
app_counts = df1['App'].value_counts()
app_counts

In [ ]:
# Filling NaN values with mean - Rating column
#df1['Rating']=df1['Rating'].fillna(df1['Rating'].mean())


In [ ]:
df1.isna().sum()

In [ ]:
# As we have negligible NaN count in Content rating, current ver and Android ver so we can drop these rows

df1=df1.dropna()


In [ ]:
df1.isna().sum().sum()

In [ ]:
# Drop duplicates
df1=df1.drop_duplicates()

In [ ]:
# Are there any non-English apps? And how to filter them?
#df2=df1['App'].str.contains('English', case=False)
#df3=df1[~df2]
#df3.head(3)

def check_english(column):
  is_eng = []
  for entry in column:
    is_eng.append(entry.isascii())
  return is_eng

num_non_english = df1.shape[0] - sum(check_english(df1['App']))
print("Number of non english apps : ", num_non_english)
df_english_apps = df1[check_english(df1['App'])]
print('Size of dataframe with only english apps : ', df_english_apps.shape)

In [ ]:
df1[~df1['App'].apply(lambda x: x.isascii())]['App']


In [ ]:
s = "AutoScout24 Switzerland – Find your new car"
for i in s:
    print(i, i.isascii())

In [ ]:
# In the size column, multiply 1,000,000 with M in the cell and multiply by 1000 if we have K in the cell.
def convert_size(Size_str):
    if 'M' in Size_str:
        return float(Size_str.replace('M', '')) * 1000000
    elif 'K' in Size_str:
        return float(Size_str.replace('K', '')) * 1000
    else:
        return np.nan

df1['Size'] = df1['Size'].apply(convert_size)
df1.head(3)

## Visualization

### Task 2: Perform the  following tasks:

##### Exercise 1: Find the number of apps in various categories by using an appropriate plot.

In [ ]:
# YOUR CODE HERE
plt.figure(figsize=(10, 6))
sns.countplot(data=df1, x='Category',order=df1['Category'].value_counts().index)
plt.title('Number of Apps in Various Categories')
plt.xlabel('Category')
plt.ylabel('Number of Apps')
plt.xticks(rotation=45)
plt.show()

##### Exercise 2: Explore the distribution of free and paid apps across different categories

**Hint:** Stacked Bar Graph, [link](https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/bar_stacked.html)

In [ ]:
# YOUR CODE HERE
category_type_counts = df1.groupby(['Category', 'Type']).size().unstack(fill_value=0)
category_type_counts.plot(kind='bar', stacked=True, figsize=(12, 8))

plt.title('Distribution of Free and Paid Apps Across Different Categories')
plt.xlabel('Category')
plt.ylabel('Number of Apps')
plt.xticks(rotation=45)
plt.legend(title='App Type')
plt.show()

##### Exercise 3: Represent the distribution of app rating on a scale of 1-5 using an appropriate plot

**Hint:** histogram / strip plot

In [ ]:
# YOUR CODE HERE
# Plotting the histogram
plt.hist(df1['Rating'], bins=range(1,6), color='skyblue', edgecolor='black')

# Adding labels and title
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.title('Distribution of App Ratings')

# Setting x-axis ticks to correspond to rating categories
plt.xticks(range(1, 6))

# Display the plot
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


##### Exercise 4: Identify outliers of the rating column by plotting the boxplot category wise and handle them.

**Hint:** Removing outliers using Z-score, quantile [link](https://kanoki.org/2020/04/23/how-to-remove-outliers-in-python/)

In [ ]:
df1.Rating.describe()

In [ ]:
# YOUR CODE HERE
mean = np.mean(df1.Rating)
std = np.std(df1.Rating)
threshold = 3
outlier = []
for i in df1.Rating:
    z = abs((i-mean)/std)
    if z > threshold:
        outlier.append(i)
print(len(outlier))

In [ ]:
sns.boxplot(df1.Rating)

In [ ]:
sns.boxplot(x="Category", y="Rating", data=df1)

##### Exercise 5: Plot the barplot of all the categories indicating no. of installs

In [ ]:
# YOUR CODE HERE
df1['Installs'] = df1['Installs'].apply(lambda x: x.replace('+',''))
df1['Installs'] = df1['Installs'].apply(lambda x: x.replace(',',''))
df1['Installs']=df1['Installs'].astype(int)

In [ ]:
sns.barplot(x="Category", y="Installs", data=df1,errorbar=None)

## Insights


### Task 3: Derive the below insights

##### Exercise 1: Does the price correlate with the size of the app?

  **Hint:** plot the scatterplot of `Size` and `Price`

In [ ]:
df1['Price'] = df1['Price'].apply(lambda x: x.replace('$',''))
df1['Price']=df1['Price'].astype(float)

In [ ]:
# YOUR CODE HERE
sns.scatterplot(x="Size", y="Price", data=df1)

##### Exercise 2: Find the popular app categories based on rating and no. of installs

**Hint:** [df.groupby.agg()](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html); Taking the average rating could be another approach



In [ ]:
df1.info()

In [ ]:
pop_app=df1.groupby('Category').agg({'Installs': 'sum', 'Rating': 'mean'})
pop_app=pop_app.sort_values(by=['Installs','Rating'],ascending=False)
pop_app.head(10)

In [ ]:
df1.groupby('Category')['Installs'].sum().sort_values(ascending=False).head(5)

In [ ]:
# YOUR CODE HERE
df1.groupby('Category')['Rating'].mean().sort_values(ascending=False).head(5)

##### Exercise 3: How many apps are produced in each year category-wise ?

  * Create a `Year` column by slicing the values of `Last Updated` column and find the Year with most no. of apps produced

    **For example**, slice the year `2017` from `February 8, 2017`

  * Find the categories which have a consistent rating in each year

      **Hint:** `sns.countplot`

In [ ]:
df1['Last Updated']=pd.to_datetime(df1['Last Updated'])

In [ ]:
df1['Last_Updated_year']=df1['Last Updated'].dt.year
df1.head(2)

In [ ]:
# YOUR CODE HERE
sns.countplot(x="Category", hue='Last_Updated_year', data=df1)

##### Exercise 4: Identify the highest paid apps with a good rating

In [ ]:
df1.App.value_counts().sort_values()

In [ ]:
df1['Reviews']=df1['Reviews'].astype(int)

In [ ]:
# YOUR CODE HERE
Highest_paid=df1.groupby('App').agg({'Price': 'mean', 'Rating': 'mean'})
Highest_paid=Highest_paid.sort_values(by=['Price','Rating'],ascending=False)
Highest_paid

##### Exercise 5: Are the top-rated apps genuine ? How about checking reviews count of top-rated apps ?

In [ ]:
# YOUR CODE HERE
top_rate=df1.groupby('App').agg({'Reviews': 'sum', 'Rating': 'mean'})
top_rate=top_rate.sort_values(by=['Reviews','Rating'],ascending=False)
top_rate

##### Exercise 6: If the number of reviews of an app is very low, what could be the reason for its top-rating ?

In [ ]:
# YOUR CODE HERE
df1[df1['Reviews']>0].groupby(['Type']).agg({'Reviews': 'mean', 'Rating': 'mean'})

In [ ]:
low_rate_high_reviews=df1.groupby('App').agg({'Reviews': 'sum', 'Rating': 'mean', 'Installs': 'sum')
low_rate_high_reviews=low_rate_high_reviews.sort_values(by=['Reviews','Rating'],ascending=[True,False])
low_rate_high_reviews

In [ ]:
low_rate_high_reviews['review_percentage']=low_rate_high_reviews['']

##### Exercise 7: What is the 95% confidence interval for the rating of apps in the Google Play Store?

In [ ]:
df1.Rating.quantile(0.95)

In [ ]:
# YOUR CODE HERE
df1[df1['Rating'] > df1.Rating.quantile(0.95)]

##### Exercise 8: Test if there is a statistically significant difference in the ratings between free and paid apps using a t-test

Steps:

* Set the null hypothesis and alternate hypothesis
* Separate the ratings of free and paid apps.
* Perform t-test: Use an independent samples t-test.
* Interpret results based on the p-value, decide whether to reject or fail to reject the null hypothesis.

In [ ]:
# YOUR CODE HERE